# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 21 2022 10:54AM,242423,15,CHR20220720,"Chadwick Pharmaceuticals, Inc.",Released
1,Jul 21 2022 10:54AM,242426,19,ACG-2102490667,ACG North America LLC,Released
2,Jul 21 2022 10:53AM,242421,15,8268913-BO,"Carwin Pharmaceutical Associates, LLC",Released
3,Jul 21 2022 10:53AM,242421,15,8268914-BO,"Carwin Pharmaceutical Associates, LLC",Released
4,Jul 21 2022 10:53AM,242421,15,8268916-BO,"Carwin Pharmaceutical Associates, LLC",Released
5,Jul 21 2022 10:53AM,242421,15,8290760-BO,"Carwin Pharmaceutical Associates, LLC",Released
6,Jul 21 2022 10:53AM,242421,15,8292177-BO,"Carwin Pharmaceutical Associates, LLC",Released
7,Jul 21 2022 10:52AM,242427,21,621121,"NBTY Global, Inc.",Released
8,Jul 21 2022 10:49AM,242348,20,6495133,"ACI Healthcare USA, Inc.",Released
9,Jul 21 2022 10:49AM,242348,20,6605147,"ACI Healthcare USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
32,242419,Released,1
33,242421,Released,5
34,242423,Released,1
35,242426,Released,1
36,242427,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
242419,NaN,1.0
242421,NaN,5.0
242423,NaN,1.0
242426,NaN,1.0
242427,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
242319,0.0,1.0
242325,0.0,1.0
242328,0.0,1.0
242336,0.0,1.0
242340,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
242319,0,1
242325,0,1
242328,0,1
242336,0,1
242340,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,242319,0,1
1,242325,0,1
2,242328,0,1
3,242336,0,1
4,242340,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,242319,,1
1,242325,,1
2,242328,,1
3,242336,,1
4,242340,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 21 2022 10:54AM,242423,15,"Chadwick Pharmaceuticals, Inc."
1,Jul 21 2022 10:54AM,242426,19,ACG North America LLC
2,Jul 21 2022 10:53AM,242421,15,"Carwin Pharmaceutical Associates, LLC"
7,Jul 21 2022 10:52AM,242427,21,"NBTY Global, Inc."
8,Jul 21 2022 10:49AM,242348,20,"ACI Healthcare USA, Inc."
86,Jul 21 2022 10:41AM,242415,10,Eywa Pharma Inc.
95,Jul 21 2022 10:34AM,242419,19,ACG North America LLC
96,Jul 21 2022 10:32AM,242418,16,Sartorius Bioprocess Solutions
97,Jul 21 2022 10:31AM,242417,22,"NBTY Global, Inc."
98,Jul 21 2022 10:13AM,242413,20,"Exact-Rx, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jul 21 2022 10:54AM,242423,15,"Chadwick Pharmaceuticals, Inc.",,1
1,Jul 21 2022 10:54AM,242426,19,ACG North America LLC,,1
2,Jul 21 2022 10:53AM,242421,15,"Carwin Pharmaceutical Associates, LLC",,5
3,Jul 21 2022 10:52AM,242427,21,"NBTY Global, Inc.",,1
4,Jul 21 2022 10:49AM,242348,20,"ACI Healthcare USA, Inc.",,78
5,Jul 21 2022 10:41AM,242415,10,Eywa Pharma Inc.,,9
6,Jul 21 2022 10:34AM,242419,19,ACG North America LLC,,1
7,Jul 21 2022 10:32AM,242418,16,Sartorius Bioprocess Solutions,,1
8,Jul 21 2022 10:31AM,242417,22,"NBTY Global, Inc.",1,
9,Jul 21 2022 10:13AM,242413,20,"Exact-Rx, Inc.",,3


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 21 2022 10:54AM,242423,15,"Chadwick Pharmaceuticals, Inc.",1,
1,Jul 21 2022 10:54AM,242426,19,ACG North America LLC,1,
2,Jul 21 2022 10:53AM,242421,15,"Carwin Pharmaceutical Associates, LLC",5,
3,Jul 21 2022 10:52AM,242427,21,"NBTY Global, Inc.",1,
4,Jul 21 2022 10:49AM,242348,20,"ACI Healthcare USA, Inc.",78,
5,Jul 21 2022 10:41AM,242415,10,Eywa Pharma Inc.,9,
6,Jul 21 2022 10:34AM,242419,19,ACG North America LLC,1,
7,Jul 21 2022 10:32AM,242418,16,Sartorius Bioprocess Solutions,1,
8,Jul 21 2022 10:31AM,242417,22,"NBTY Global, Inc.",,1
9,Jul 21 2022 10:13AM,242413,20,"Exact-Rx, Inc.",3,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 21 2022 10:54AM,242423,15,"Chadwick Pharmaceuticals, Inc.",1,
1,Jul 21 2022 10:54AM,242426,19,ACG North America LLC,1,
2,Jul 21 2022 10:53AM,242421,15,"Carwin Pharmaceutical Associates, LLC",5,
3,Jul 21 2022 10:52AM,242427,21,"NBTY Global, Inc.",1,
4,Jul 21 2022 10:49AM,242348,20,"ACI Healthcare USA, Inc.",78,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jul 21 2022 10:54AM,242423,15,"Chadwick Pharmaceuticals, Inc.",1.0,NaN
1,Jul 21 2022 10:54AM,242426,19,ACG North America LLC,1.0,NaN
2,Jul 21 2022 10:53AM,242421,15,"Carwin Pharmaceutical Associates, LLC",5.0,NaN
3,Jul 21 2022 10:52AM,242427,21,"NBTY Global, Inc.",1.0,NaN
4,Jul 21 2022 10:49AM,242348,20,"ACI Healthcare USA, Inc.",78.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 21 2022 10:54AM,242423,15,"Chadwick Pharmaceuticals, Inc.",1.0,0.0
1,Jul 21 2022 10:54AM,242426,19,ACG North America LLC,1.0,0.0
2,Jul 21 2022 10:53AM,242421,15,"Carwin Pharmaceutical Associates, LLC",5.0,0.0
3,Jul 21 2022 10:52AM,242427,21,"NBTY Global, Inc.",1.0,0.0
4,Jul 21 2022 10:49AM,242348,20,"ACI Healthcare USA, Inc.",78.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2423914,137.0,19.0
12,242336,1.0,0.0
15,484844,6.0,0.0
16,1454450,9.0,0.0
19,1211926,16.0,2.0
20,484761,81.0,0.0
21,1938917,8.0,0.0
22,242417,0.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2423914,137.0,19.0
1,12,242336,1.0,0.0
2,15,484844,6.0,0.0
3,16,1454450,9.0,0.0
4,19,1211926,16.0,2.0
5,20,484761,81.0,0.0
6,21,1938917,8.0,0.0
7,22,242417,0.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,137.0,19.0
1,12,1.0,0.0
2,15,6.0,0.0
3,16,9.0,0.0
4,19,16.0,2.0
5,20,81.0,0.0
6,21,8.0,0.0
7,22,0.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,137.0
1,12,Released,1.0
2,15,Released,6.0
3,16,Released,9.0
4,19,Released,16.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21,22
Status,,,,,,,,
Executing,19.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0
Released,137.0,1.0,6.0,9.0,16.0,81.0,8.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21,22
0,Executing,19.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0
1,Released,137.0,1.0,6.0,9.0,16.0,81.0,8.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21,22
0,Executing,19.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0
1,Released,137.0,1.0,6.0,9.0,16.0,81.0,8.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()